<a href="https://colab.research.google.com/github/smesindianou/homeworks/blob/main/Exercise7_to_vima_teasers_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests # για να φέρνουμε τον html κώδικα των ιστοσελίδων
from bs4 import BeautifulSoup # για να βρίσκουμε μέσα στον html κώδικα, τα περιεχόμενα που μας ενδιαφέρουν
import pandas as pd # για να βάζουμε τα δεδομένα που συλλέγουμε σε dataframes

import time # για να κάνουμε παύσεις ανάμεσα στα αιτήματα που κάνουμε στον server του site και να μην τον ρίξουμε ή να μην μας αποκλείσει
import random # για να δημιουργούμε ποικιλία στον χρόνο των παύσεων ώστε τα αιτήματα στον server να μοιάζουν με ανθρώπινα και όχι μηχανικά

In [ ]:
def get_webpage_soup(url):
  # Ορισμός των headers για το requests
  headers = {
        "User-Agent": "Mozilla/5.0"
    }
  # Αίτημα στο server να φέρει τον html κώδικα από το url
  response = requests.get(url, headers=headers)

  # Έλεγχος της κατάστασης της σελίδας
  if response.status_code != 200:
    print(f"Error: {response.status_code}")
    return None
  else:
    # Μετατροπή του πηγαίου κώδικα της ιστοσελίδας σε αναζητήσιμο κείμενο (soup)
    print(f"{url} was scraped successfully")
    return BeautifulSoup(response.text, 'html.parser')

In [ ]:
def get_teaser_elements_from_articlesL(articlesL):
  articles_dataL = [] # κενή λίστα, στην οποία θα κάνουμε append() το dictionary με τα data κάθε άρθρου
  for article in articlesL:
    try:
     title = article.find("h3", {"class" : "o-head f-400 my-0 is-size-2 zonabold"}).text.strip()
    except:
      title=""
    try:
      url = article.find("a", {"class": "columns is-mobile is-multiline"})["href"]
    except:
      url=""
    try:
      datetime_str = article.find("time").text.strip()
      date_str = datetime_str.split(",")[0]
      time_str = datetime_str.split(",")[1].strip()
    except:
      date_str = ""
      time_str = ""
    try:
      author = article.find("span", {"class":"vima-author line-height-1 manrope is-size-5 notblack-c f-700"}).text.strip()
    except:
      author = ""
    try:
      excert = article.find("p", {"class":"is-size-normal mb-0 mt-4 text-ellipsis text-ellipsis-4 grey-c line-height-1-3"}).text.strip()
    except:
      excert = ""

    # Δημιουργία λεξικού με τα data του άρθρου
    articleD = {"title": title, "article_url": url, "date": date_str, "time": time_str, "author": author, "description": excert}

    # Προσθήκη των data του κάθε άρθρου στη λίστα των data των άρθρων
    articles_dataL.append(articleD)

  return articles_dataL

In [ ]:
def scrape_to_vima_teaser_pages(archive_name, category, start_page_nbr, end_page_nbr):
  # Κενή αρχική λίστα
  all_articles_dataL = []

  print(f"\rScraping started", end="")

  for page_nbr in range(start_page_nbr, end_page_nbr+1): # βάζω +1 για να σκραπάρω και τον αριθμό του τελευταίου page που έχω ορίσει
    # η δομή των teasers pages urls του kathmerini.gr
    url = f"https://www.tovima.gr/{archive_name}/{category}/page/{page_nbr}/"

    print(f"\rScraping page {page_nbr}: {url}" + " "*20, end="")

    # φέρνω τη σούπα της σελίδας με το function που έφτιαξα παραπάνω στο βήμα 2
    soup = get_webpage_soup(url)

    # βάζω τον κώδικα με τον οποίο βρίσκω όλα τα blocks των άρθρων - το έφτιαξα στο βήμα 6
    articlesL = soup.find_all("div", {"class":"column is-full wrap-category-row"})

    # εξορύσσω τα δεδομένα όλων των άρθρων της σελίδας με το function που έφτιαξα στο βήμα 8
    page_articles_dataL = get_teaser_elements_from_articlesL(articlesL)

    # προσθέτω τα δεδομένα των άρθρων της σελίδας στη λίστα με τα δεδομένα όλων των άρθρων - τη δημιούργησα στην αρχή αυτού του function
    all_articles_dataL.extend(page_articles_dataL)

    # Τυχαία καθυστέρηση μεταξύ 1 και 3 δευτερολέπτων ανάμεσα στα page scrapes
    delay = random.uniform(1, 3)
    time.sleep(delay)

  # Δημιουργία dataframe από τη λίστα με τα λεξικά των δεδομένων κάθε άρθρου
  df = pd.DataFrame(all_articles_dataL)
  print(f"\rScraping completed. Fetshed {len(df)} articles.")

  return df

In [ ]:
# π.χ. 1:
to_vima_latest_news_df = scrape_to_vima_teaser_pages ("latest-news","", 1, 2)
to_vima_latest_news_df

Scraping page 1: https://www.tovima.gr/latest-news//page/1/                    https://www.tovima.gr/latest-news//page/1/ was scraped successfully
Scraping page 2: https://www.tovima.gr/latest-news//page/2/                    https://www.tovima.gr/latest-news//page/2/ was scraped successfully
Scraping completed. Fetshed 40 articles.


,title,article_url,date,time,author,description
0,Παίκτες της Παρτιζάν συμμετείχαν σε συμπλοκή έ...,https://www.tovima.gr/2025/05/06/sports/paikte...,06.05.2025,00:32,,Άσχημες εικόνες εκτυλίχθηκαν έξω από νυχτερινό...
1,Ο Τραμπ δίνει λεφτά σε παράτυπους μετανάστες γ...,https://www.tovima.gr/2025/05/05/world/o-tramp...,05.05.2025,23:49,,Ο λόγος που ο Ντόναλντ Τραμπ επέλεξε να προχωρ...
2,Επεισόδιο με πυροβολισμούς στον Εύοσμο μετά απ...,https://www.tovima.gr/2025/05/05/society/thess...,05.05.2025,23:18,,Η ΕΛΑΣ έχει εξαπολύσει ανθρωποκυνηγητό για τον...
3,Μεγάλοι νικητές στα βραβεία Pulitzer η Wall St...,https://www.tovima.gr/2025/05/05/media/vraveia...,05.05.2025,22:54,,Τα θέματα που απέσπασαν τα βραβεία Pulitzer γι...
4,Νέες αποκαλύψεις για τον υποκινητή της βομβιστ...,https://www.tovima.gr/2025/05/05/society/thess...,05.05.2025,22:39,,Πού έχει επικεντρώσει τις έρευνές της η ΕΛΑΣ -...
5,Το Ισραήλ απάντησε στους Χούθι με μπαράζ βομβα...,https://www.tovima.gr/2025/05/05/world/to-isra...,05.05.2025,22:10,,Οι στόχοι των χτυπημάτων του Ισραήλ - Σε συναγ...
6,Μαρτυρίες από το blackout στην Ισπανία - «Κυκλ...,https://www.tovima.gr/print/world/martyries-lf...,05.05.2025,22:00,,Αναζητούνται ακόμα τα αίτια που βύθισαν στο σκ...
7,Κόντρα της AfD με την BfV για το «εξακριβωμένα...,https://www.tovima.gr/2025/05/05/world/germani...,05.05.2025,21:42,,Η απόφαση της Ομοσπονδιακής Υπηρεσίας Προστασί...
8,Αντίσταση στον Τραμπ - Τα παραδείγματα του Καν...,https://www.tovima.gr/print/world/antistasi-st...,05.05.2025,21:27,Μαρίλια Παπαθανασίου,"Ο Μάρκ Κάρνεϊ κέρδισε στον Καναδά, ο Αντονι Αλ..."
9,OT FORUM – Στουρνάρας: Η ΕΚΤ θα συνεχίσει τις ...,https://www.tovima.gr/2025/05/05/finance/ot-fo...,05.05.2025,21:15,,Τι είπε ο διοικητής της Τράπεζας της Ελλάδος Γ...


In [ ]:
# π.χ. 2:
to_vima_politics_df = scrape_to_vima_teaser_pages("category","politics", 1, 2)
to_vima_politics_df

Scraping page 1: https://www.tovima.gr/category/politics/page/1/                    https://www.tovima.gr/category/politics/page/1/ was scraped successfully
Scraping page 2: https://www.tovima.gr/category/politics/page/2/                    https://www.tovima.gr/category/politics/page/2/ was scraped successfully
Scraping completed. Fetshed 32 articles.


,title,article_url,date,time,author,description
0,Φαραντούρης: Νέα παρέμβαση για Ομάδα Αλήθειας ...,https://www.tovima.gr/2025/05/05/politics/fara...,05.05.2025,16:21,,Ο Νικόλας Φαραντούρης κάλεσε την Ευρωπαϊκή Επι...
1,"Kids Wallet, κοινωνική εργασία, τεχνικά γυμνάσ...",https://www.tovima.gr/2025/05/05/society/parav...,05.05.2025,15:56,,Αναλυτικά η δομή και τα τέσσερα κεφάλαια του ε...
2,Ξανά στους ελληνικούς αιθέρες το θρυλικό T-6 H...,https://www.tovima.gr/2025/05/05/politics/pole...,05.05.2025,14:05,,Η εντυπωσιακή τελετή ένταξης του αεροσκάφους τ...
3,Εκτός «Νίκης» ο Ν. Παπαδόπουλος - «Να παραδώσε...,https://www.tovima.gr/2025/05/05/politics/niki...,05.05.2025,12:55,,Μετά τον βανδαλισμό στην Εθνική Πινακοθήκη - Π...
4,Live η εξειδίκευση των μέτρων για την αντιμετώ...,https://www.tovima.gr/2025/05/05/politics/live...,05.05.2025,12:53,,Στην εξειδίκευση των μέτρων για την ανήλικη βί...
5,Βορίδης από Χίο: Μείωση στις ροές μεταναστών- ...,https://www.tovima.gr/2025/05/05/politics/vori...,05.05.2025,12:28,,Ο Βορίδης και τι είπε από την Χίο για τους μετ...
6,Παπαστεργίου: Ετσι θα λειτουργεί το Kids Walle...,https://www.tovima.gr/2025/05/05/politics/papa...,05.05.2025,12:20,,«Το Kids Wallet είναι μια απλή εφαρμογή γονικο...
7,Μητσοτάκης για βία ανηλίκων: Απαιτεί μεθοδική ...,https://www.tovima.gr/2025/05/05/politics/mits...,05.05.2025,12:02,,«Προϊόν βιωματικής προσέγγισης η Εθνική Στρατη...
8,Live η παρουσίαση της Εθνικής Στρατηγικής κατά...,https://www.tovima.gr/2025/05/05/politics/live...,05.05.2025,10:18,,Στις 12:30 η εξειδίκευση των μέτρων για τη βία...
9,Εθνική Στρατηγική για τη βία ανηλίκων: Τα νέα ...,https://www.tovima.gr/2025/05/05/politics/ethn...,05.05.2025,09:46,,Η Εθνική Στρατηγική παρουσιάζεται σήμερα σε σύ...


In [ ]:
# π.χ. 3:
to_vima_tempi_df = scrape_to_vima_teaser_pages("search","ΤΕΜΠΗ", 1, 2)
to_vima_tempi_df

Scraping page 1: https://www.tovima.gr/search/ΤΕΜΠΗ/page/1/                    https://www.tovima.gr/search/ΤΕΜΠΗ/page/1/ was scraped successfully
Scraping page 2: https://www.tovima.gr/search/ΤΕΜΠΗ/page/2/                    https://www.tovima.gr/search/ΤΕΜΠΗ/page/2/ was scraped successfully
Scraping completed. Fetshed 40 articles.


,title,article_url,date,time,author,description
0,Τσουκαλάς για Τέμπη: Επιχειρείται ομερτά για ν...,https://www.tovima.gr/2025/05/04/politics/tsou...,04.05.2025,17:46,,"«Αυτό πρέπει να σπάσει, αν θέλουμε να βγει η α..."
1,ΣΥΡΙΖΑ για Τέμπη: Ως πότε θα κρύβει την αλήθει...,https://www.tovima.gr/2025/05/03/politics/syri...,03.05.2025,18:59,,«Σε δύο τεχνικές εκθέσεις που παραδόθηκαν στην...
2,«Για ποιο έγκλημα πρέπει να ανιχνευθεί η ενδεχ...,https://www.tovima.gr/2025/05/03/opinions/temp...,03.05.2025,11:57,Γρηγόρης Καλφέλης,"«Ελπίζουμε ακόμη για την αλήθεια, αν δεν επικρ..."
3,"Τέμπη: «Το τρένο έπιασε φωτιά» Η κλήση στο ""10...",https://www.tovima.gr/2025/05/02/society/tempi...,02.05.2025,07:57,,Νέα ηχητικά στοιχεία για τα Τέμπη. Η κλήση στο...
4,Φαραντούρης: Ζήτησε συνεννόηση για το κατηγορη...,https://www.tovima.gr/2025/05/01/politics/temp...,01.05.2025,17:16,,"Μια ενιαία στάση των προοδευτικών κομμάτων, με..."
5,Τέμπη: Γιατί πήγε τον Ιούνιο η συζήτηση στην Ε...,https://www.tovima.gr/2025/04/30/politics/temp...,30.04.2025,19:35,,Αναβολή 40 ημερών πήρε το αίτημα για τα Τέμπη ...
6,Τέμπη: «Ανοιξε» η δικογραφία - Τι ζήτησε η Ζωή...,https://www.tovima.gr/2025/04/30/politics/temp...,30.04.2025,10:53,Ελένη Γκρήγκοβιτς,"Οι βουλευτές που το επιθυμούν, θα μπορούν από ..."
7,Στη Βουλή η δικογραφία για τα Τέμπη: Γιατί τώρα;,https://www.tovima.gr/2025/04/30/podcasts/to-v...,30.04.2025,06:58,,"Η Ιωάννα Μάνδρου, δικαστική συντάκτρια στο «ΒΗ..."
8,ΣΥΡΙΖΑ: Σκληρή ρητορική πριν από τη νέα μετωπι...,https://www.tovima.gr/2025/04/29/politics/syri...,29.04.2025,19:55,Γιάννης Ζωιτός,"Γιατί ο ΣΥΡΙΖΑ δεν πείθεται, σε καμία περίπτωσ..."
9,Μαρινάκης για Τέμπη: Το μοντέλο Τριαντοπούλου ...,https://www.tovima.gr/2025/04/29/politics/mari...,29.04.2025,13:12,Ελένη Γκρήγκοβιτς,Ο Παύλος Μαρινάκης σχολίασε ενδεικτικά ότι πρέ...


In [ ]:
to_vima_latest_news_df.to_csv('/content/drive/MyDrive/Datasets/to_vima_latest_news.csv', index=False)

In [ ]:
data_entry = pd.read_csv("/content/drive/MyDrive/Datasets/to_vima_latest_news.csv")
data_entry

,title,article_url,date,time,author,description
0,Παίκτες της Παρτιζάν συμμετείχαν σε συμπλοκή έ...,https://www.tovima.gr/2025/05/06/sports/paikte...,06.05.2025,00:32,NaN,Άσχημες εικόνες εκτυλίχθηκαν έξω από νυχτερινό...
1,Ο Τραμπ δίνει λεφτά σε παράτυπους μετανάστες γ...,https://www.tovima.gr/2025/05/05/world/o-tramp...,05.05.2025,23:49,NaN,Ο λόγος που ο Ντόναλντ Τραμπ επέλεξε να προχωρ...
2,Επεισόδιο με πυροβολισμούς στον Εύοσμο μετά απ...,https://www.tovima.gr/2025/05/05/society/thess...,05.05.2025,23:18,NaN,Η ΕΛΑΣ έχει εξαπολύσει ανθρωποκυνηγητό για τον...
3,Μεγάλοι νικητές στα βραβεία Pulitzer η Wall St...,https://www.tovima.gr/2025/05/05/media/vraveia...,05.05.2025,22:54,NaN,Τα θέματα που απέσπασαν τα βραβεία Pulitzer γι...
4,Νέες αποκαλύψεις για τον υποκινητή της βομβιστ...,https://www.tovima.gr/2025/05/05/society/thess...,05.05.2025,22:39,NaN,Πού έχει επικεντρώσει τις έρευνές της η ΕΛΑΣ -...
5,Το Ισραήλ απάντησε στους Χούθι με μπαράζ βομβα...,https://www.tovima.gr/2025/05/05/world/to-isra...,05.05.2025,22:10,NaN,Οι στόχοι των χτυπημάτων του Ισραήλ - Σε συναγ...
6,Μαρτυρίες από το blackout στην Ισπανία - «Κυκλ...,https://www.tovima.gr/print/world/martyries-lf...,05.05.2025,22:00,NaN,Αναζητούνται ακόμα τα αίτια που βύθισαν στο σκ...
7,Κόντρα της AfD με την BfV για το «εξακριβωμένα...,https://www.tovima.gr/2025/05/05/world/germani...,05.05.2025,21:42,NaN,Η απόφαση της Ομοσπονδιακής Υπηρεσίας Προστασί...
8,Αντίσταση στον Τραμπ - Τα παραδείγματα του Καν...,https://www.tovima.gr/print/world/antistasi-st...,05.05.2025,21:27,Μαρίλια Παπαθανασίου,"Ο Μάρκ Κάρνεϊ κέρδισε στον Καναδά, ο Αντονι Αλ..."
9,OT FORUM – Στουρνάρας: Η ΕΚΤ θα συνεχίσει τις ...,https://www.tovima.gr/2025/05/05/finance/ot-fo...,05.05.2025,21:15,NaN,Τι είπε ο διοικητής της Τράπεζας της Ελλάδος Γ...
